# String Matching in Python (pandas)

By the end of this tutorial you will know:

 " how to approximately match strings and determine how similar they are by going over various examples ? "
 


file 1 : 

In [1]:
import pandas as pd
import psycopg2


master = pd.read_csv('master_kabkota.csv')
master.head(3)

,id_kab,kab_kota,provinsi
0,3372,KOTA SURAKARTA,Jawa Tengah
1,3506,KAB. KEDIRI,Jawa Timur
2,7313,KAB. WAJO,Sulawesi Selatan


File 2 : 

In [2]:
coord = pd.read_csv('polusi_kabkota.csv')
coord.head(3)

,name,province,district_city,kab_kota,country,coordinates_latitude,coordinates_longitude,current_ts,current_condition,current_humidity
0,Banda Aceh,Aceh,Kota Banda Aceh,Banda Aceh,Indonesia,5.54167,95.33333,18/03/2020 05:00,Broken clouds,56
1,Langsa,Aceh,Kota Langsa,Langsa,Indonesia,4.46830,97.96830,18/03/2020 16:00,Broken clouds,56
2,Lhokseumawe,Aceh,Kota Lhokseumawe,Lhokseumawe,Indonesia,5.18010,97.15070,18/03/2020 16:00,Rain,66


case nya di sini kita akan mencocokan dua file, untuk mengambil semua informasi di table nya yaitu id_kab, coordinates_latitude dan coordinates_longitude 

ket : 
1. master : 514 row
2. coord : 3440 row

In [3]:
coord['kab_kota']=coord['kab_kota'].str.upper()  #mengubah format kolom kab_kota menjadi UPPERCASE
df = coord.merge(master, on='kab_kota', how='left') #merge 2 dataframe menggunakan kab_kota
df.head(3)

,name,province,district_city,kab_kota,country,coordinates_latitude,coordinates_longitude,current_ts,current_condition,current_humidity,id_kab,provinsi
0,Banda Aceh,Aceh,Kota Banda Aceh,BANDA ACEH,Indonesia,5.54167,95.33333,18/03/2020 05:00,Broken clouds,56,NaN,NaN
1,Langsa,Aceh,Kota Langsa,LANGSA,Indonesia,4.46830,97.96830,18/03/2020 16:00,Broken clouds,56,NaN,NaN
2,Lhokseumawe,Aceh,Kota Lhokseumawe,LHOKSEUMAWE,Indonesia,5.18010,97.15070,18/03/2020 16:00,Rain,66,NaN,NaN


In [4]:
df['id_kab'].isnull().sum() #jumlah kolom id_kab yang null

3440

tenyata dari 3440 data, tidak ada satupun nama kab_kota yang matching, oke dri pola nya kelihatan bahwa df1 masih terdapat kata "kab" dan "kota", maka proses selanjutnya adalah cleansing data nya (df1) 

In [5]:
master['kab_kota'] = master['kab_kota'].str.replace(r'(KAB. |KOTA )','')
master.head(3)

,id_kab,kab_kota,provinsi
0,3372,SURAKARTA,Jawa Tengah
1,3506,KEDIRI,Jawa Timur
2,7313,WAJO,Sulawesi Selatan


In [6]:
coord['kab_kota']=coord['kab_kota'].str.upper()  #mengubah format kolom kab_kota menjadi UPPERCASE
df = coord.merge(master, on='kab_kota', how='left') #merge 2 dataframe menggunakan kab_kota
df = df.drop_duplicates()
df.head(3)

,name,province,district_city,kab_kota,country,coordinates_latitude,coordinates_longitude,current_ts,current_condition,current_humidity,id_kab,provinsi
0,Banda Aceh,Aceh,Kota Banda Aceh,BANDA ACEH,Indonesia,5.54167,95.33333,18/03/2020 05:00,Broken clouds,56,1171.0,Aceh
1,Langsa,Aceh,Kota Langsa,LANGSA,Indonesia,4.46830,97.96830,18/03/2020 16:00,Broken clouds,56,1174.0,Aceh
2,Lhokseumawe,Aceh,Kota Lhokseumawe,LHOKSEUMAWE,Indonesia,5.18010,97.15070,18/03/2020 16:00,Rain,66,1173.0,Aceh


In [7]:
df['id_kab'].isnull().sum() #jumlah kolom id_kab yang null

387

setelah dicleansing data nya kita ulangi lagi proses sebelumnya, dan voila terdapat 387 kab_kota yang blm termatching, baiklah coba kita lihat datanya seperti apa 

In [8]:
df_notnull = df[df['id_kab'].notnull()] #df yang id_kab nya tidak null
df1 = df[df['id_kab'].isnull()] #df yang id_kab nya null 
df1.head(3)

,name,province,district_city,kab_kota,country,coordinates_latitude,coordinates_longitude,current_ts,current_condition,current_humidity,id_kab,provinsi
46,Mukomuko,Bengkulu,Kabupaten Mukomuko,MUKOMUKO,Indonesia,-2.568900,101.11180,18/03/2020 16:00,Rain,76,NaN,NaN
49,Jakarta,DKI Jakarta,Jakarta,JAKARTA,Indonesia,-6.236704,106.79324,18/03/2020 17:00,Scattered clouds,83,NaN,NaN
55,Siulak,Jambi,Kabupaten Kerinci,KERINCI,Indonesia,-1.946670,101.33417,18/03/2020 16:00,Rain,95,NaN,NaN


In [9]:
#import module 
from fuzzywuzzy import process

#define list 
kab_similarity=[]
similarity=[]
prov_similarity=[]
id_kab_similarity=[]

for i in df1.kab_kota:
    Ratios = process.extract(i, master.kab_kota, limit=1 )
    kab_similarity.append(Ratios[0][0])
    prov_similarity.append(master.loc[master['kab_kota'] == Ratios[0][0], 'provinsi'].iloc[0])
    similarity.append(Ratios[0][1]) 
    id_kab_similarity.append(master.loc[master['kab_kota'] == Ratios[0][0], 'id_kab'].iloc[0])

c:\python27\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
df1['kab_similarity']=kab_similarity
df1['id_kab_similarity'] = id_kab_similarity
df1['prov_similarity']=prov_similarity
df1['similarity']=similarity
# df1
df2 = df1[['province','kab_kota','kab_similarity','id_kab_similarity','prov_similarity','similarity']]
df2.head(3)

c:\python27\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\python27\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\python27\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

,province,kab_kota,kab_similarity,id_kab_similarity,prov_similarity,similarity
46,Bengkulu,MUKOMUKO,MUKO MUKO,1706,Bengkulu,94
49,DKI Jakarta,JAKARTA,ADM. JAKARTA TIMUR,3175,DKI Jakarta,90
55,Jambi,KERINCI,KERINCI,1501,Jambi,100


terlihat hasil nya, ternyata beberpa typo yng ditemukan adalah salah penggunaan spasi. Dan telihat angka hasil similarity nya > 90%, artinya hasilnya similarity nya optimal, lanjut untuk menggabungkan dengan table sebelumnya 

In [11]:
df3 = df1[['name','province','district_city','kab_kota','country','coordinates_latitude','coordinates_longitude','current_ts','current_condition','current_humidity','id_kab_similarity','prov_similarity']]
df3.head(3)

,name,province,district_city,kab_kota,country,coordinates_latitude,coordinates_longitude,current_ts,current_condition,current_humidity,id_kab_similarity,prov_similarity
46,Mukomuko,Bengkulu,Kabupaten Mukomuko,MUKOMUKO,Indonesia,-2.568900,101.11180,18/03/2020 16:00,Rain,76,1706,Bengkulu
49,Jakarta,DKI Jakarta,Jakarta,JAKARTA,Indonesia,-6.236704,106.79324,18/03/2020 17:00,Scattered clouds,83,3175,DKI Jakarta
55,Siulak,Jambi,Kabupaten Kerinci,KERINCI,Indonesia,-1.946670,101.33417,18/03/2020 16:00,Rain,95,1501,Jambi


In [12]:
df3 = df3.rename(columns={'id_kab_similarity':'id_kab','prov_similarity':'provinsi'})

In [13]:
result = []
result = pd.concat([df_notnull,df3],sort=False)
result.head(3)

,name,province,district_city,kab_kota,country,coordinates_latitude,coordinates_longitude,current_ts,current_condition,current_humidity,id_kab,provinsi
0,Banda Aceh,Aceh,Kota Banda Aceh,BANDA ACEH,Indonesia,5.54167,95.33333,18/03/2020 05:00,Broken clouds,56,1171.0,Aceh
1,Langsa,Aceh,Kota Langsa,LANGSA,Indonesia,4.46830,97.96830,18/03/2020 16:00,Broken clouds,56,1174.0,Aceh
2,Lhokseumawe,Aceh,Kota Lhokseumawe,LHOKSEUMAWE,Indonesia,5.18010,97.15070,18/03/2020 16:00,Rain,66,1173.0,Aceh


In [14]:
result['id_kab'].isnull().sum() #jumlah kolom id_kab yang null

0

DONE !!